In [1]:
]activate .

  Activating project at `~/Amazon WorkDocs Drive/My Documents/P2/QuantumOptics.jl/example`


In [2]:
]st

Status `~/Amazon WorkDocs Drive/My Documents/P2/QuantumOptics.jl/example/Project.toml`
  [f6369f11] ForwardDiff v0.10.34
  [429524aa] Optim v1.7.4
  [6e0679c1] QuantumOptics v1.0.8 `..`
  [295af30f] Revise v3.4.0


In [3]:
#import Revise
using QuantumOptics
import ForwardDiff as FD
import Optim

# System

In [4]:
# 3 level kerr transmon with drive
ba = FockBasis(1)
ω0 = 5.0
α = -0.2
T2 = 1e4
function get_Ht(p::Vector{<:Tp}) where Tp
    A, freq, ϕ, T = p
    op = 2π*([number(ba), 2\create(ba)*number(ba)*destroy(ba), im*(create(ba)-destroy(ba))])
    fun = [t->Tp(ω0), t->Tp(α), t->A*cospi(2t*freq + 2ϕ)*sinpi(t/T)^2]
    H_at_t = LazySum([f(zero(eltype(p))) for f=fun], op)
    function Ht(t,u)
        for k=1:length(fun)
            H_at_t.factors[k] = fun[k](t)
        end
        H_at_t
    end
    return Ht
end

get_Ht (generic function with 1 method)

In [5]:
ψ0 = Operator(SpinBasis(1/2), basisstate(ba, 1), basisstate(ba, 2))
target = ψ0*exp(im*0.5π*dense(sigmax(SpinBasis(1/2))))
function cost(par)
    T = par[4]
    Ht = get_Ht(par)
    ts = (0.0, T)
    _, ψT = timeevolution.schroedinger_dynamic(ts, ψ0, Ht)
    1-abs2(tr(target'last(ψT))/2)*exp(-T/T2)
end

cost (generic function with 1 method)

# initial states are the basis state

In [6]:
# short times, works ok
p0 = let T = 1e-3
    [0.01, 5, 0.25, T]
end
cost(p0)

1.0

In [7]:
FD.gradient(cost, p0)

4-element Vector{Float64}:
 -4.1751998733606453e-16
 -8.34772811023868e-19
  2.1531396173854207e-16
 -1.2523414972596036e-14

In [8]:
# long times
## dynamics works ok
## gradient gets NaN
p0 = let T = 100
    [0.01, 5, 0.25, T]
end
cost(p0)

0.010652603278478057

In [9]:
FD.gradient(cost, p0)

4-element Vector{Float64}:
 NaN
 NaN
 NaN
 NaN

# initial states are random states

In [10]:
ψ0 = Operator(SpinBasis(1/2), randstate(ba), randstate(ba))

Operator(dim=2x2)
  basis left:  Fock(cutoff=1)
  basis right: Spin(1/2)
 0.359256+0.436229im  0.529682+0.104348im
 0.605568+0.560291im  0.409226+0.735584im

In [11]:
# short times, works ok
p0 = let T = 1e-3
    [0.01, 5, 0.25, T]
end
cost(p0)

0.5637903463230448

In [12]:
FD.gradient(cost, p0)

4-element Vector{Float64}:
 -5.979609706978611e-6
 -1.1957577187575068e-8
 -2.4251158416139668e-5
 -3.4803176814634202

In [13]:
# long times, works ok
p0 = let T = 100
    [0.01, 5, 0.25, T]
end
cost(p0)

0.5141118524433785

In [14]:
FD.gradient(cost, p0)

4-element Vector{Float64}:
 63.68995771065536
 62.38740146319189
  0.5331138800982694
  1.3439324603867508

In [15]:
function fg!(L,G,p)
    if G != nothing
        G .= FD.gradient(cost, p)
        any(isnan.(G)) && error("NaN !!")
    end
    cost(p)
end

fg! (generic function with 1 method)

In [16]:
#Optim.optimize(Optim.only_fg!(fg!), p0, Optim.Options(show_trace=true))